# SEIRD Model First Example

In this notebook we present how to use the `epimodels` module to set up an instantiation of the SEIRD model, using some toy data.

In [1]:
# Load necessary libraries
import numpy as np
import epimodels as em
import matplotlib
import plotly.graph_objects as go
from matplotlib import pyplot as plt
from iteration_utilities import deepflatten

## Model Setup
### Define setup matrices for the SEIRD Model

In [2]:
# Populate the model
regions = ['London', 'Cornwall']

### Set the parameters and initial conditions of the model and bundle everything together

In [3]:
# Instantiate model
model = em.SEIRDModel()

# Set the region names, contact and regional data of the model
model.set_regions(regions)

# Set ICs parameters
ICs = em.SEIRDICs(
    model=model,
    susceptibles_IC=[1500, 700],
    exposed_IC=[0, 0],
    infectives_IC=[40, 32],
    recovered_IC=[0, 0],
    dead_IC=[0, 0]
)

# Set transmission parameters
transmission_parameters = em.SEIRDTransmission(
    model=model,
    beta=10,
    kappa=0.2,
    gamma=0.1,
    Pd=0.05
)

# Set other simulation parameters
simulation_parameters = em.SEIRDSimParameters(
    model=model,
    region_index=2,
    method='RK45',
    times=np.arange(1, 20.5, 0.5).tolist()
)

# Set all parameters in the controller
parameters = em.SEIRDParametersController(
    model=model,
    ICs=ICs,
    transmission_parameters=transmission_parameters,
    simulation_parameters=simulation_parameters
)

### Simulate for one of the regions: **London**

In [4]:
# Simulate for all the regions
outputs = []

for r, reg in enumerate(regions):
    # List of initial conditions and parameters that characterise the model
    parameters.simulation_parameters.region_index = r + 1

    # Simulate using the ODE solver
    outputs.append(model.simulate(parameters))

## Plot the comparments of the two methods against each other
### Setup ``plotly`` and default settings for plotting

In [5]:
colours = ['blue', 'red', 'green', 'purple', 'orange', 'black', 'gray', 'pink']

### Plot the comparments of the two methods against each other

In [6]:
# Trace names - represent the solver used for the simulation
trace_name = ['region {}'.format(r) for r in regions]

# Compartment list - type and age
comparments = []
for n in model.output_names():
    comparments.append('{}'.format(n))

# Plot for each comparment
for c, comparment in enumerate(comparments):
    fig = go.Figure()
    for o, out in enumerate(outputs):
        fig.add_trace(
            go.Scatter(
                y=out[:, c],
                x=parameters.simulation_parameters.times,
                mode='lines',
                name=trace_name[o],
                line_color=colours[o]
            )
        )

    # Add axis labels
    fig.update_layout(
        boxmode='group',
        title=comparment, 
        width=800,
        plot_bgcolor='white',
        xaxis=dict(linecolor='black'),
        yaxis=dict(linecolor='black'),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ))

    fig.show()